In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments

2023-07-17 10:08:50.020107: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 10:08:50.219681: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-17 10:08:51.093098: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-07-17 10:08:51.093205: W tensorflow/

In [2]:
max_seq_length = 2048

## 读取模型

In [3]:
model_name = "baichuan-inc/Baichuan-13B-Chat"


# QLoRa更进一步，引入了4位量化、双量化和利用nVidia统一内存进行分页。

# 简而言之，QLoRa工作原理如下:

# 1.4位NormalFloat量化:这是一种改进量化的方法。它确保每个量化仓中有相同数量的值。这避免了计算问题和异常值的错误。
# 2.双量化:QLoRa的作者将其定义如下“对量化常量再次量化以节省额外内存的过程。”
# 3.统一内存分页:它依赖于NVIDIA统一内存管理，自动处理CPU和GPU之间的页到页传输。它可以保证GPU处理无错，特别是在GPU可能耗尽内存的情况下。

# 所有这些步骤都大大减少了微调所需的内存，同时性能几乎与标准微调相当。
# (https://avoid.overfit.cn/post/4c4c86e3f7974157a7a8e81c57a0f8a4)


quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_use_double_quant=True,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_compute_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_fast=False,
                                          trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=quant_config,
                                             device_map="auto",
                                             torch_dtype=torch.float16,
                                             trust_remote_code=True)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## peft的qlora设置

In [4]:
import bitsandbytes as bnb
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)


def find_all_linear_names(model):
    """
    找出所有全连接层，为所有全连接添加adapter
    """
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

target_modules = find_all_linear_names(model)

config = LoraConfig(r=64,
                    lora_alpha=16,
                    lora_dropout=0.05,
                    target_modules=target_modules,
                    bias="none",
                    task_type="CAUSAL_LM")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 223150080 || all params: 7183488000 || trainable%: 3.106430747848399


## 查看模型中参数的类型

In [5]:
from collections import defaultdict

def verify_model_dtype(model):
    """
    查看模型种各种类型的参数的情况
    """
    dtype2param_num = defaultdict(int)  # 每种数据类型的参数量
    dtype2param_name = defaultdict(list)  # 每种数据类型的参数名称
    dtype2trainable_param_num = defaultdict(int)  # 每种数据类型参与训练的参数量
    dtype2trainable_param_name = defaultdict(list)  # 每种数据类型参与训练的参数名称
    for name, p in model.named_parameters():
        dtype = p.dtype
        dtype2param_num[dtype] += p.numel()
        dtype2param_name[dtype].append(name)
        if p.requires_grad:
            dtype2trainable_param_num[dtype] += p.numel()
            dtype2trainable_param_name[dtype].append(name)
    # 统计全部参数中，各种类型参数分布
    total = 0
    print('verify all params of the model')
    for k, v in dtype2param_num.items():
        total += v
    for k, v in dtype2param_num.items():
        print(k, v, v / total)
    for k, v in dtype2trainable_param_name.items():
        print(k, v)

    print()
    # 统计可训练参数中，各种类型参数分布
    print('verify trainable params the model')
    total_trainable = 0
    for k, v in dtype2trainable_param_num.items():
        total_trainable += v
    for k, v in dtype2trainable_param_num.items():
        print(k, v, v / total_trainable)
    for k, v in dtype2trainable_param_num.items():
        print(k, v)

verify_model_dtype(model)

verify all params of the model
torch.float32 878924800 0.12235348621728052
torch.uint8 6304563200 0.8776465137827195
torch.float32 ['base_model.model.model.layers.0.self_attn.W_pack.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.W_pack.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.W_pack.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.W_pack.lora_B.default.weight', 'b

## 测试用data

In [6]:
train_data = [
    {
        "human": "为什么 Midjourney 效果远远好于开源的 Stable Diffusion Model?",
        "assistant": "因为题主不会用SD,从门外汉角度得出了错误结论。 Midjourney特点是新手友好,但可控性差、细节优化难。下限高，但上限低。张张都精致唬人，但你想要调节细节时，会发现越调越歪，哪哪都不对劲，抽盲盒一样。"
    },
    {
        "human": "核酸检测机构需要什么资质",
        "assistant": "市卫健委将审核申报资料，并结合市区新型冠状病毒核酸检测需求进行综合评估。经评估，具备相应资质和条件的，纳入我市开展新型冠状病毒核酸检测机构名单，并通知进行新型冠状病毒实验活动备案。"
    },
]*50
train_data

[{'human': '为什么 Midjourney 效果远远好于开源的 Stable Diffusion Model?',
  'assistant': '因为题主不会用SD,从门外汉角度得出了错误结论。 Midjourney特点是新手友好,但可控性差、细节优化难。下限高，但上限低。张张都精致唬人，但你想要调节细节时，会发现越调越歪，哪哪都不对劲，抽盲盒一样。'},
 {'human': '核酸检测机构需要什么资质',
  'assistant': '市卫健委将审核申报资料，并结合市区新型冠状病毒核酸检测需求进行综合评估。经评估，具备相应资质和条件的，纳入我市开展新型冠状病毒核酸检测机构名单，并通知进行新型冠状病毒实验活动备案。'},
 {'human': '为什么 Midjourney 效果远远好于开源的 Stable Diffusion Model?',
  'assistant': '因为题主不会用SD,从门外汉角度得出了错误结论。 Midjourney特点是新手友好,但可控性差、细节优化难。下限高，但上限低。张张都精致唬人，但你想要调节细节时，会发现越调越歪，哪哪都不对劲，抽盲盒一样。'},
 {'human': '核酸检测机构需要什么资质',
  'assistant': '市卫健委将审核申报资料，并结合市区新型冠状病毒核酸检测需求进行综合评估。经评估，具备相应资质和条件的，纳入我市开展新型冠状病毒核酸检测机构名单，并通知进行新型冠状病毒实验活动备案。'},
 {'human': '为什么 Midjourney 效果远远好于开源的 Stable Diffusion Model?',
  'assistant': '因为题主不会用SD,从门外汉角度得出了错误结论。 Midjourney特点是新手友好,但可控性差、细节优化难。下限高，但上限低。张张都精致唬人，但你想要调节细节时，会发现越调越歪，哪哪都不对劲，抽盲盒一样。'},
 {'human': '核酸检测机构需要什么资质',
  'assistant': '市卫健委将审核申报资料，并结合市区新型冠状病毒核酸检测需求进行综合评估。经评估，具备相应资质和条件的，纳入我市开展新型冠状病毒核酸检测机构名单，并通知进行新型冠状病毒实验活动备案。'},
 {'human': '为什么 Midjourney 效果远远

In [7]:
import json
from torch.utils.data import Dataset


class SFTDataset(Dataset):
    def __init__(self, file, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.bos_token_id = tokenizer.bos_token_id
        self.eos_token_id = tokenizer.eos_token_id
        self.eos_token = tokenizer.eos_token
        self.bos_token = tokenizer.bos_token
        self.max_seq_length = max_seq_length
#         with open(file, 'r', encoding='utf8') as f:
#             data_list = f.readlines()
        data_list = train_data
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        # 每条数据格式为: <s>input1</s>target1</s>input2</s>target2</s>...
#         print(self.data_list)
        data = self.data_list[index]
#         data = json.loads(data)
#         conversation = data['conversation']
        conversation = self.data_list[index]

        # 收集多轮对话
        utterances = []
        utterances.append(conversation['human'])
        utterances.append(conversation['assistant'])
        utterances_ids = self.tokenizer(utterances, add_special_tokens=False).input_ids
#         print(utterances_ids)
        # 模型的输入格式为：<s>input1</s>target1</s>input2</s>target2</s>...
        input_ids = [self.bos_token_id]
        target_mask = [0]  # 用于对input进行mask，只计算target部分的loss
        for i, utterances_id in enumerate(utterances_ids):
            input_ids += (utterances_id + [self.eos_token_id])
            if i % 2 == 0:
                target_mask += [0] * (len(utterances_id) + 1)
            else:
                target_mask += [1] * (len(utterances_id) + 1)
        assert len(input_ids) == len(target_mask)
        # 对长度进行截断
        input_ids = input_ids[:self.max_seq_length]
        target_mask = target_mask[:self.max_seq_length]
        attention_mask = [1] * len(input_ids)
        assert len(input_ids) == len(target_mask) == len(attention_mask)
        inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'target_mask': target_mask
        }
        return inputs

In [8]:
train_dataset = SFTDataset("dummyfilename", tokenizer, max_seq_length)
train_dataset.__getitem__(0)

{'input_ids': [1,
  13701,
  6944,
  31156,
  4443,
  31106,
  4353,
  23099,
  31213,
  31235,
  31237,
  31445,
  31135,
  867,
  914,
  743,
  2812,
  6566,
  9749,
  81,
  2,
  23258,
  20266,
  2753,
  31204,
  13720,
  31125,
  31382,
  31435,
  31357,
  31976,
  10150,
  31252,
  5093,
  9282,
  20817,
  73,
  6944,
  31156,
  4443,
  8662,
  31161,
  27029,
  28216,
  31125,
  31354,
  31197,
  31529,
  31309,
  31965,
  76,
  11081,
  9784,
  31597,
  73,
  31214,
  31614,
  31229,
  72,
  31354,
  31185,
  31614,
  31664,
  73,
  31632,
  31632,
  31228,
  16585,
  35815,
  31167,
  72,
  31354,
  31203,
  5138,
  13980,
  11081,
  31183,
  72,
  31190,
  2742,
  31615,
  31499,
  31615,
  34014,
  72,
  31853,
  31853,
  6225,
  31209,
  32844,
  72,
  32482,
  33189,
  32918,
  3494,
  73,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [9]:
from typing import Any, Dict, List
import torch


class SFTDataCollator(object):
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.pad_token_id = tokenizer.pad_token_id

    def __call__(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        # 找出batch中的最大长度
        lengths = [len(x['input_ids']) for x in batch]
        # 取出batch中的最大长度，如果超过max_seq_length，则取max_seq_length
        batch_max_len = min(max(lengths), self.max_seq_length)
        # batch_max_len = self.max_seq_length

        input_ids_batch, attention_mask_batch, target_mask_batch = [], [], []
        # truncate and padding
        for x in batch:
            input_ids = x['input_ids']
            attention_mask = x['attention_mask']
            target_mask = x['target_mask']
            padding_len = batch_max_len - len(input_ids)
            # padding
            input_ids = input_ids + [self.pad_token_id] * padding_len
            attention_mask = attention_mask + [0] * padding_len
            target_mask = target_mask + [0] * padding_len
            # truncate
            input_ids = input_ids[:self.max_seq_length]
            attention_mask = attention_mask[:self.max_seq_length]
            target_mask = target_mask[:self.max_seq_length]

            input_ids_batch.append(input_ids)
            attention_mask_batch.append(attention_mask)
            target_mask_batch.append(target_mask)

        # 将list转换为tensor，得到最终的的模型输入
        input_ids_batch = torch.tensor(input_ids_batch, dtype=torch.long)
        attention_mask_batch = torch.tensor(attention_mask_batch, dtype=torch.long)
        target_mask_batch = torch.tensor(target_mask_batch, dtype=torch.long)
        inputs = {
            'input_ids': input_ids_batch,
            'attention_mask': attention_mask_batch,
            'target_mask': target_mask_batch
        }
        return inputs
    
data_collator = SFTDataCollator(tokenizer, max_seq_length)

## 训练

In [13]:
import torch
import torch.nn as nn


class Loss(object):
    """
    所有loss的类父类
    """

    def __call__(self, model, inputs, training_args, return_outputs=False):
        """
        todo label smoothing
        用于计算loss。
        看源码发现，return_outputs=True为train时调用，return_outputs=False为eval和predict调用
        :param model: 模型
        :param inputs: 模型输入，dict
        :param training_args: 训练配置参数
        :param return_outputs:是否返回模型的输出
        :return:
        """
        raise NotImplemented


class TargetLMLoss(Loss):

    def __init__(self, ignore_index):
        super().__init__()
        self.ignore_index = ignore_index
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=ignore_index)

    def __call__(self, model, inputs, training_args, return_outputs=False):
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        target_mask = inputs['target_mask']
        # 模型前馈预测
        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        return_dict=True)
        logits = outputs["logits"] if isinstance(outputs, dict) else outputs[0]

        # 将labels中不属于target的部分，设为ignore_index，只计算target部分的loss
        labels = torch.where(target_mask == 1, input_ids, self.ignore_index)
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        # Flatten the tokens
        loss = self.loss_fn(shift_logits.view(-1, shift_logits.size(-1)),
                            shift_labels.view(-1))
        return (loss, outputs) if return_outputs else loss


loss_func = TargetLMLoss(ignore_index=tokenizer.pad_token_id)

In [15]:
import transformers
from transformers import (
    PreTrainedModel,
    TrainingArguments,
    DataCollator,
    PreTrainedTokenizerBase,
    EvalPrediction,
    TrainerCallback,
)
from typing import Callable, Dict, List, Optional, Tuple, Union, Any
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers.utils import (
    logging,
)
from typing import Optional
import os
import torch


logger = logging.get_logger(__name__)

# Name of the files used for checkpointing
TRAINING_ARGS_NAME = "training_args.bin"
TRAINER_STATE_NAME = "trainer_state.json"
OPTIMIZER_NAME = "optimizer.pt"
SCHEDULER_NAME = "scheduler.pt"
SCALER_NAME = "scaler.pt"


class Trainer(transformers.Trainer):
    """
    主要修改逻辑：通过传入compute_loss，支持自定义loss计算方式
    """
    def __init__(
            self,
            model: Union[PreTrainedModel, nn.Module] = None,
            args: TrainingArguments = None,
            data_collator: Optional[DataCollator] = None,
            train_dataset: Optional[Dataset] = None,
            eval_dataset: Optional[Dataset] = None,
            tokenizer: Optional[PreTrainedTokenizerBase] = None,
            model_init: Callable[[], PreTrainedModel] = None,
            compute_metrics: Optional[Callable[[EvalPrediction], Dict]] = None,
            callbacks: Optional[List[TrainerCallback]] = None,
            optimizers: Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR] = (None, None),
            preprocess_logits_for_metrics: Callable[[torch.Tensor, torch.Tensor], torch.Tensor] = None,
            compute_loss=None,
    ):
        super(Trainer, self).__init__(
            model=model,
            args=args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=tokenizer,
            model_init=model_init,
            compute_metrics=compute_metrics,
            callbacks=callbacks,
            optimizers=optimizers,
            preprocess_logits_for_metrics=preprocess_logits_for_metrics,
        )
        self.loss_func = compute_loss

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        重写loss的计算方式
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        return self.loss_func(model, inputs, self.args, return_outputs)


class LoRATrainer(Trainer):
    """
    修改checkkpoint的保存逻辑，只保存lora
    """
    def _save(self, output_dir: Optional[str] = None, state_dict=None):
        # If we are executing this function, we are the process zero, so we don't check for that.
        output_dir = output_dir if output_dir is not None else self.args.output_dir
        os.makedirs(output_dir, exist_ok=True)
        logger.info(f"Saving model checkpoint to {output_dir}")
        # 保存lora权重和配置
        self.model.save_pretrained(
            output_dir, state_dict=state_dict, safe_serialization=self.args.save_safetensors
        )

        if self.tokenizer is not None:
            self.tokenizer.save_pretrained(output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))

In [16]:
training_args = TrainingArguments("output",
                                  do_train=True,
                                  per_device_train_batch_size=1,
                                  gradient_accumulation_steps=10,
                                  learning_rate=1e-4,
                                  lr_scheduler_type="constant_with_warmup",
                                  warmup_steps=0,
                                  max_steps=100,
                                  logging_steps=20,
                                  remove_unused_columns=False,
                                  seed=114514,
                                  data_seed=1919810,
                                  fp16=True,
                                  group_by_length=False,
                                  dataloader_pin_memory=False,
                                  save_strategy='steps',
                                  save_steps=20,
                                  save_total_limit=3)

trainer = LoRATrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_loss=loss_func
)

In [17]:
trainer.train()

Step,Training Loss
20,2.192800


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_15236/4032920361.py:1 in <module>                                          │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_15236/4032920361.py'                 │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:1661 in train              │
│                                                                                           │
│   1658 │   │   inner_training_loop = find_executable_batch_size(                          │
│   1659 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batc │
│   1660 │   │   )                                                                          │
│ ❱ 1661 │   │   return inner_training_loop(                                                │
│   1662 │   │   │   args=args,                                                             │
│   1663 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                         │
│   1664 │   │   │   trial=trial,                                                           │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:1946 in                    │
│ _inner_training_loop                                                                      │
│                                                                                           │
│   1943 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self. │
│   1944 │   │   │   │                                                                      │
│   1945 │   │   │   │   with self.accelerator.accumulate(model):                           │
│ ❱ 1946 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)               │
│   1947 │   │   │   │                                                                      │
│   1948 │   │   │   │   if (                                                               │
│   1949 │   │   │   │   │   args.logging_nan_inf_filter                                    │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:2756 in training_step      │
│                                                                                           │
│   2753 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)             │
│   2754 │   │                                                                              │
│   2755 │   │   with self.compute_loss_context_manager():                                  │
│ ❱ 2756 │   │   │   loss = self.compute_loss(model, inputs)                                │
│   2757 │   │                                                                              │
│   2758 │   │   if self.args.n_gpu > 1:                                                    │
│   2759 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training │
│                                                                                           │
│ /tmp/ipykernel_15236/2697024621.py:72 in compute_loss                                     │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_15236/2697024621.py'                 │
│                                                                                           │
│ /tmp/ipykernel_15236/3189755451.py:36 in __call__                                         │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykern